In [2]:
import re
import time
import os
import pickle
import datetime

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import anndata
import scanpy as sc
import h5py
import yaml


In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
#sc.settings.set_figure_params(dpi=120)

The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata     0.7.5
scanpy      1.6.0
sinfo       0.3.4
-----
PIL                         8.4.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
bottleneck                  1.3.2
cached_property             1.5.2
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
dunamai                     1.11.0
entrypoints                 0.3
get_version     

In [4]:
ROOT_DIR = ".."
CONFIG = os.path.join(ROOT_DIR, "config.yaml")


In [18]:
with open(CONFIG, "r") as filehandle:
    config = yaml.load(filehandle, yaml.SafeLoader)
backgrounds = {k: None for k in config["background"]}
DATA_DIR = os.path.join(ROOT_DIR, config["data_dir"])
RESULTS_DIR = os.path.join(ROOT_DIR, config["results_dir"])

for back in backgrounds:
    path=os.path.join(DATA_DIR, "backgrounds", f"{back}.csv")
    backgrounds[back] = pd.read_csv(path)

In [19]:
RESULTS_DIR

'../2022-04-30_results_wake_vs_sleep/'

In [6]:
backgrounds

{'KC':   cluster     louvain_res  idx
 0      ab  louvain_res2.0    4
 1     abp  louvain_res2.0   19
 2       y  louvain_res3.0    0,
 'glia':   cluster     louvain_res  idx
 0     CXG  louvain_res2.0   76
 1     ALG  louvain_res8.0   19
 2      PG  louvain_res4.0   81
 3    EG_1  louvain_res4.0   28
 4    EG_2  louvain_res8.0   64}

In [7]:
adata = sc.read_h5ad(os.path.join(DATA_DIR, "h5ad/Preloom/adata_25042022.h5ad"))

In [8]:
adata.obs.columns

Index(['Age', 'Condition', 'Genotype', 'Run', 'Singlet', 'Sleep_Stage',
       'Treatment', 'doublet_scores', 'n_counts', 'n_genes', 'percent_mito',
       'predicted_doublets', 'louvain', 'louvain_res0.4', 'louvain_res0.8',
       'louvain_res1.0', 'louvain_res1.2', 'louvain_res1.6', 'louvain_res2.0',
       'louvain_res3.0', 'louvain_res4.0', 'louvain_res8.0', 'Run_Set',
       'Cluster_ID_res2', 'Cluster_ID_res8', 'Neurotransmitter', 'all_res8',
       'HRG_sqrt', 'CRG_sqrt'],
      dtype='object')

In [9]:
# cluster_to_res_dict = {
#     'CXG': ['louvain_res2.0' , str(76)],
#     'ALG+': ['louvain_res8.0' , str(19)],
#     'ALG-': ['louvain_res8.0' , str(133)],
#     'PNG': ['louvain_res4.0' , str(81)],
#     #'SPG': ['louvain_res4.0' , str(77)],
#     'EGN--': ['louvain_res2.0' , str(18)],
#     'EGN+-': ['louvain_res4.0' , str(28)],
#     'EGN++': ['louvain_res8.0' , str(64)],
#     'ab': ['louvain_res2.0' , str(4)],
#     'abp': ['louvain_res2.0' , str(19)],
#     'y': ['louvain_res3.0' , str(0)],
#     'nonPAM': ['louvain_res8.0' , str(136)],
#     'PAM': ['louvain_res8.0' , str(37)],
#     'clock': ['louvain_res8.0' , str(84)],
#     'Tyr': ['louvain_res3.0' , str(105)],
#     'Oct': ['louvain_res3.0' , str(103)],
#     '5-HT': ['louvain_res8.0' , str(151)],
#     'ring_A': ['louvain_res8.0' , str(100)],
#     'ring_B': ['louvain_res8.0' , str(95)],     
#     'adPN': ['louvain_res3.0' , str(31)],
#     #'T4T5': ['louvain_res2.0' , str(38)],
#     'PB45': ['louvain_res2.0' , str(43)],
#     #'Tm1TmY8': ['louvain_res3.0' , str(101)],
#     #'Mi1': ['louvain_res3.0' , str(91)],
#     #'T1': ['louvain_res8.0' , str(135)]
# }

In [10]:
comparisons =[]
CELLTYPE_COL = "Cluster_ID_res8"
for back in backgrounds:
    all_clusters = backgrounds[back]["cluster"]
    for cluster in all_clusters:
        comparison_name = f"{cluster}_vs_rest"
        other_clusters = [c for c in all_clusters if c != cluster]
        adata.obs[comparison_name] = None
        adata.obs.loc[adata.obs[CELLTYPE_COL] == cluster, comparison_name] = "target"
        adata.obs.loc[adata.obs[CELLTYPE_COL].isin(other_clusters), comparison_name] = "background"
        comparisons.append(
            (comparison_name, ["target"], "background", None)
        ) 

print(comparisons)
        
# comparisons =[
#     ('annotated_res8', ['EG_1'], 'ring_B', None),
# ]
# ('annotated_res8', ['EG_1'], 'ring_B', None),

[('ab_vs_rest', ['target'], 'background', None), ('abp_vs_rest', ['target'], 'background', None), ('y_vs_rest', ['target'], 'background', None), ('CXG_vs_rest', ['target'], 'background', None), ('ALG_vs_rest', ['target'], 'background', None), ('PG_vs_rest', ['target'], 'background', None), ('EG_1_vs_rest', ['target'], 'background', None), ('EG_2_vs_rest', ['target'], 'background', None)]


In [11]:
# adata.obs.loc[adata.obs["PG_vs_rest"] == "background", CELLTYPE_COL]

In [12]:
all_results = {}

In [13]:
from collections import Counter

In [16]:
# Set the resolution to use here
cluster_obs = CELLTYPE_COL
results = {}

for comparison in comparisons:
    obs, groups, reference, subset = comparison
    comp_name = f'{"_".join(groups)}_vs_{reference}'
    results = {}

    results[comp_name] = {}
    all_reference = ["target"]
    all_reference = ["background"]         

    for n, z in enumerate(zip(all_groups, all_reference)):
        if n != 0 or subset != None:
            obs_idx = 'within_cluster_diff'
        else:
            obs_idx = obs[:]

        print(f'Comparing {z[0]} vs {z[1]} under {obs_idx}')

        adata.obs['current_diff'] = None

        if type(z[1]) == list:
            ref_cells = set()
            for g in z[1]:
                ref_cells = ref_cells.union(adata.obs[adata.obs[obs_idx] == g].index)
        else:
            ref_cells = adata.obs[adata.obs[obs_idx] == z[1]].index

        test_cells = set()
        for g in z[0]:
            test_cells = test_cells.union(adata.obs[adata.obs[obs_idx] == g].index)

        adata.obs.loc[ref_cells, 'current_diff'] = 'ref'
        adata.obs.loc[test_cells, 'current_diff'] = 'test'



        try:
            sc.tl.rank_genes_groups(adata, 'current_diff', groups=['test'], reference='ref' , method='wilcoxon', n_genes=adata.raw.shape[1])
        except IndexError:
            print('\tNot enough cells in one group')
            continue
        except ZeroDivisionError:
            print('\tZero division!')
            continue
        except ValueError:
            print('\tSample doesn\'t contain one group')
            print(Counter(adata.obs['current_diff']))
            continue

        group_n = sum(adata.obs[obs_idx].isin(z[0]))

        genes = np.ravel(adata.uns['rank_genes_groups']['names'].tolist())

        group_exp_n = (adata[adata.obs[obs_idx].isin(z[0])].raw[:, genes].X > 0).sum(axis=0)
        if type(z[1]) == list:
            ref_n = sum(adata.obs[obs_idx].isin(z[1]))
            ref_exp_n = (adata[adata.obs[obs_idx].isin(z[1])].raw[:, genes].X > 0).sum(axis=0)
        else:
            ref_n = sum(adata.obs[obs_idx] == z[1])
            ref_exp_n = (adata[adata.obs[obs_idx] == z[1]].raw[:, genes].X > 0).sum(axis=0)


        group_frac = (group_exp_n/group_n)
        ref_frac = (ref_exp_n/ref_n)

        adata.uns['rank_genes_groups']['basemean'] = np.ravel((adata.raw[:, genes].X).mean(axis=0))
        adata.uns['rank_genes_groups']['cells_group'] = np.ravel(group_exp_n)
        adata.uns['rank_genes_groups']['cells_ref'] = np.ravel(ref_exp_n)
        adata.uns['rank_genes_groups']['fraction_group'] = np.ravel(group_frac)
        adata.uns['rank_genes_groups']['fraction_ref'] = np.ravel(ref_frac)

        if n == 0 and subset == None:
            results[comp_name]['All_Cells'] = adata.uns['rank_genes_groups']
        else:
            results[comp_name][f'{"_".join(z[0])}_vs_{z[1]}'] = adata.uns['rank_genes_groups']


    if subset == None: 
        all_results[f"{obs}__{'All_Combined_No_ZT2_Wake'}__{'_'.join(groups)}_vs_{reference}"] = results
    else:
        all_results[f"{obs}__{'All_Combined_No_ZT2_Wake'}__{all_groups[0][0]}_vs_{all_reference[0]}"] = results            


Comparing ['target'] vs background under ab_vs_rest
ranking genes


... storing 'current_diff' as categorical


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:05)
Comparing ['target'] vs background under abp_vs_rest
ranking genes


... storing 'current_diff' as categorical


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:05)
Comparing ['target'] vs background under y_vs_rest
ranking genes


...

    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
Comparing ['target'] vs background under EG_1_vs_rest
ranking genes


... storing 'current_diff' as categorical


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
Comparing ['target'] vs background under EG_2_vs_rest
ranking genes


... storing 'current_diff' as categorical


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)


In [56]:
def generate_markers_file(data, marker):
    
    data = data[["logfoldchange","pval"]]
    data.reset_index(inplace=True)
    data["gene"] = data["index"]
    data["avg_logFC"] = data["logfoldchange"]
    
    data = data[["gene", "avg_logFC", "pval"]]
    
    dest_dir = os.path.join(DATA_DIR, "markers")
    
    os.makedirs(dest_dir, exist_ok=True)
    dest_file = os.path.join(dest_dir, f"{marker}.tsv")
    data.to_csv(dest_file, sep="\t", index=False)

In [58]:
COMPARISONS_DIR=f"{RESULTS_DIR}/comparisons"
os.makedirs(COMPARISONS_DIR, exist_ok=True)


for c, i in all_results.items():
    
    # if c == "abp_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background":
    #     break
    for _, data in i.items():
        # c = '__'.join(c.split("__")[1:])
        print(f'Working on {c}')
        
        filename = f"{datetime.date.today().strftime('%Y%m%d')}_{c}_DiffExp_UpAndDown"
        excel_file = os.path.join(COMPARISONS_DIR, f"{filename}.xlsx")
        csv_file = os.path.join(COMPARISONS_DIR, f"{filename}.csv")
        
                     
        with pd.ExcelWriter(excel_file) as writer:        
            for n, comparison in enumerate(sorted(data.keys(), key=lambda x: (int(x.split('_')[0].split(' ')[-1]) if x.split('_')[0].split(' ')[-1].isdigit() else float('-inf')))):
                print(f'\tOn: {comparison}')
                df = pd.DataFrame(index=np.ravel(data[comparison]['names'].tolist()), columns =['basemean', 'cells_group', 'fraction_group', 'cells_ref', 'fraction_ref', 'zscore', 'logfoldchange', 'pval', 'pval_adj'])
                df['basemean'] = data[comparison]['basemean']
                df['cells_group'] = data[comparison]['cells_group']
                df['cells_ref'] = data[comparison]['cells_ref']
                df['fraction_group'] = data[comparison]['fraction_group']
                df['fraction_ref'] = data[comparison]['fraction_ref']
                df['zscore'] = np.ravel(data[comparison]['scores'].tolist())
                df['logfoldchange'] = np.ravel(data[comparison]['logfoldchanges'].tolist())
                df['pval'] = np.ravel(data[comparison]['pvals'].tolist())
                df['pval_adj'] = np.ravel(data[comparison]['pvals_adj'].tolist())
                if n == 0:
                    sheet_name = 'All Cells'
                else:
                    try:
                        sheet_name = f"Cluster {int(comparison.split('__')[0]) + 1}"
                    except:
                        sheet_name = f"Cluster {comparison.split('__')[0]}"
                df.to_excel(writer, sheet_name=sheet_name)
                df.to_csv(csv_file)
                marker=re.search("(.*)_vs_.*__.*_vs.*", c).group(1)
                print(marker)
                generate_markers_file(df, marker=marker)

Working on ab_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
ab


/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/vibflysleep/anaconda3/envs/SleepML/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Working on abp_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
abp
Working on y_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
y
Working on CXG_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
CXG
Working on ALG_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
ALG
Working on PG_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
PG
Working on EG_1_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
EG_1
Working on EG_2_vs_rest__All_Combined_No_ZT2_Wake__target_vs_background
	On: All_Cells
EG_2


In [12]:
for sample, data in adatas.items():
    data.detect_marker_genes(n_genes = data.adata.raw.shape[1])  

NameError: name 'adatas' is not defined